In [1]:
import pandas as pd 
df=pd.read_csv("user_reviews2.csv") 
df['Sentiment'] = [1 if x > 3 else 0 for x in df.Rating]
df.head()

,Unnamed: 0,ID1,Name,ID2,Topic,Review,Rating,Date,Sentiment
0,0,Dz49XFTtSbUtN1cgfZ-9uA,Karen C.,OIdyZ4tieuG0YBbMVjDtcA,Yamo,This is the most awful place I've ever set foo...,1,31-07-2014,0
1,1,ARXQsXEfamGNn5OCpPosiQ,Mike L.,k9GG1gFhQAQSQAYydjUjSQ,Mikado Sushi,Did happy hour here today. Five stars for bein...,5,04-06-2017,1
2,2,Ho47TvoR9I-ZxwMAMliwZg,John V.,DEeJ2qcDGZGBS8KJsJGC8w,All Good Pizza,I love the pizzas & sandwiches & beer garden v...,5,03-09-2018,1
3,3,FI0dcCAuFmW0lutdKO2UwA,Kyle K.,ix9rWyVohyRoAtARuTNOaw,Columbus Cafe,Bartenders here have big tudes but otherwise i...,2,07-07-2009,0
4,4,Ejgl_zozeJCbTTJyuB1Azw,Nicholas H.,VCjSMI5G99_lPRzichTBhg,Sushi Hon,Came to Sushi Hon as a late Valentine's Day di...,3,21-02-2018,0


In [6]:
df=df[["Review","Sentiment"]]
df.head()

,Review,Sentiment
0,This is the most awful place I've ever set foo...,0
1,Did happy hour here today. Five stars for bein...,1
2,I love the pizzas & sandwiches & beer garden v...,1
3,Bartenders here have big tudes but otherwise i...,0
4,Came to Sushi Hon as a late Valentine's Day di...,0


In [7]:
df['Sentiment'].value_counts()

1    6016
0    6016
Name: Sentiment, dtype: int64

In [8]:
from bs4 import BeautifulSoup
import re
                  
def utility(text):
    text=BeautifulSoup(text,"html.parser").get_text()
    text=re.sub('\[[^]]*\]', '',text)
    pattern=r'[^a-zA-z0-9\s.]'
    text=re.sub(pattern,'',text)
    return text
                  
df['Review']=df['Review'].apply(utility)
df['Review']=df['Review'].str.lower()
df.head()

,Review,Sentiment
0,this is the most awful place ive ever set foot...,0
1,did happy hour here today. five stars for bein...,1
2,i love the pizzas sandwiches beer garden vib...,1
3,bartenders here have big tudes but otherwise i...,0
4,came to sushi hon as a late valentines day din...,0


In [9]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

df['Review']=df['Review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df.head()

,Review,Sentiment
0,awful place ive ever set foot in. still cannot...,0
1,happy hour today. five stars one best values city,1
2,love pizzas sandwiches beer garden vibe. reall...,1
3,bartenders big tudes otherwise ok place. cool ...,0
4,came sushi hon late valentines day dinner so. ...,0


In [10]:
df['Review'][0]

'awful place ive ever set foot in. still cannot understand health department hasnt shut yet. gave chance even though place complete dump huge mistake. besides rusty kitchenware sticky menus lack ventilation service worst ive ever seen. lady taking orders rudest person ive ever met. rush order kept pushing check face didnt even get food yet eating there. another guy ordering go asked eat food flat said late. poor guy ended standing outside restaurant eating food. anyways sat boyfriend half hour watching people ordered us get food first. minutes later saw started cooking food completely fed up. got left. walked place good mood walked extremely irritated. ive never felt need write bad review restaurant even though bad experiences now.do eat completely disgusting service terrible food look good all.'

#### Word2Vec

In [11]:
tmp_corpus = df['Review'].map(lambda x: x.split('.'))

In [13]:
from tqdm import tqdm
# corpus [[w1,w2,w3..],[..]]
corpus = []
for i in tqdm(range(len(tmp_corpus))):
    for line in tmp_corpus[i]:
        words = [x for x in line.split()]
        corpus.append(words)

100%|█████████████████████████████████████████████████████████████████████████| 12032/12032 [00:00<00:00, 21093.01it/s]


In [14]:
num_of_sentences = len(corpus)
num_of_words = 0
for line in corpus:
    num_of_words += len(line)

print('Num of sentences - %s'%(num_of_sentences))
print('Num of words - %s'%(num_of_words))

Num of sentences - 114743
Num of words - 722489


In [16]:
from gensim.models import Word2Vec
from gensim.models import Phrases
from gensim.models.phrases import Phraser

phrases = Phrases(sentences=corpus,min_count=25,threshold=50)
bigram = Phraser(phrases)

In [17]:
for index,sentence in enumerate(corpus):
    corpus[index] = bigram[sentence]

In [18]:
size = 100
window_size = 2 # sentences weren't too long, so
epochs = 50
min_count = 2
workers = 4

# train word2vec model using gensim
model = Word2Vec(corpus, sg=1,window=window_size,size=size,
                 min_count=min_count,workers=workers,iter=epochs,sample=0.01)

In [25]:
import random
def shuffle_corpus(sentences):
    shuffled = list(sentences)
    random.shuffle(shuffled)
    return shuffled

model.build_vocab(sentences=shuffle_corpus(corpus),update=True)

for i in range(5):
    model.train(sentences=shuffle_corpus(corpus),epochs=5,total_examples=model.corpus_count)

In [26]:
# save model
model.save('w2v_model')

In [31]:
model.most_similar(positive=['awful'], topn=5)

C:\Users\Raju\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('hideously', 0.5824334621429443),
 ('bad', 0.5338049530982971),
 ('deplorable', 0.4887511134147644),
 ('floored', 0.4826033115386963),
 ('tainted', 0.47940629720687866)]